In [ ]:
%matplotlib inline

import importlib

import matplotlib.pyplot as plt
import numpy as np

import metropolis_ising
importlib.reload(metropolis_ising)

ising = metropolis_ising.MetropolisIsing(lattice_size=4, bond_energy=1, temperature=3, initial_temperature="hi", sweeps=10000)
ising.show_lattice(show_ticks=False)

In [ ]:
ising.metropolis()

ising.plot_magnetization()
ising.plot_energy()

In [ ]:
# Data gets normalized immediatly.
equilibrium = 2000
equilibrium_energy = ising.energy_history[equilibrium:] / ising.no_of_sites
equilibrium_magnetization = ising.magnet_history[equilibrium:] / ising.no_of_sites

In [ ]:
print(ising.binning_method(equilibrium_energy, 12, "Energy", True))
print(ising.binning_method(np.absolute(equilibrium_magnetization), 12, "Magnetization", True))
print(ising.binning_method((equilibrium_magnetization*ising.no_of_sites)**2, 12, "Magnetization Squared", True))

In [ ]:
magnet_correlation_time, _ = ising.numpy_autocorrelation(equilibrium_magnetization)
print("Magnetization correlation time is {0} Monte Carlo Sweeps.".format(magnet_correlation_time))

energy_correlation_time, _ = ising.numpy_autocorrelation(equilibrium_energy, False)
print("Energy correlation time is {0} Monte Carlo Sweeps.".format(energy_correlation_time))

In [ ]:
ising.sample_every_two_correlation_times(equilibrium_energy, equilibrium_magnetization, energy_correlation_time)

In [ ]:
def binning_temperature_range():
    energys = []
    energy_errors = []
    energy_correlations = []
    
    magnetizations = []
    magnetization_errors = []
    magnetization_correlations = []
    
    for temperature in np.arange(0.2, 5, 0.2):
        ising = metropolis_ising.MetropolisIsing(lattice_size=4, bond_energy=1, temperature=temperature, initial_temperature="hi", sweeps=10000)
        ising.metropolis()
        energy_correlation_time, energy, energy_error = ising.binning_method(ising.energy_history[2000:] / ising.no_of_sites, 10, "Energy")
        magnetization_correlation_time, magnetization, magnetization_error = ising.binning_method(np.absolute(ising.magnet_history[5000:] / ising.no_of_sites), 10, "Energy")
        
        print(energy, energy_error)
        energys.append((temperature, energy))
        energy_errors.append((temperature, energy_error))
        energy_correlations.append((temperature, energy_correlation_time))
        magnetizations.append((temperature, np.absolute(magnetization)))
        magnetization_errors.append((temperature, magnetization_error))
        magnetization_correlations.append((temperature, magnetization_correlation_time))
    
    return energys, energy_errors, energy_correlations, magnetizations, magnetization_errors, magnetization_correlations

In [ ]:
energy, energy_error, energy_correlation, magnetization, magnetization_error, magnetization_correlation = binning_temperature_range()

In [ ]:
ising.plot_correlation_time_range(energy_correlation, 4, "Energy")
ising.plot_correlation_time_range(magnetization_correlation, 4, "Magnetization")
ising.plot_quantity_range(energy, energy_error, "Energy", 4)
ising.plot_quantity_range(magnetization, magnetization_error, "Magnetization", 4, exact=ising.exact_magnetization(0.2, 5))

In [ ]:
def temperature_range(lower, upper, step, lattice_size_L):
    """Calculate energy and magnetization over a range of temperatures."""
    energy = []
    energy_error = []
    
    magnetization = []
    magnetization_error = []
    
    specific_heat = []
    specific_heat_error = []
    
    correlation_times = []
    
    for temperature in np.arange(lower, upper, step):
        ising = metropolis_ising.MetropolisIsing(lattice_size_L=lattice_size_L, bond_energy_J=1, temperature_T=temperature,
                                                 initial_temperature="lo", sweeps=10000)
        
        ising.metropolis()
        equilibrium = 2000

        energy_history = ising.energy_history[equilibrium:] / ising.no_of_sites
        magnet_history = ising.magnet_history[equilibrium:] / ising.no_of_sites
        correlation_time, acfunc = ising.numpy_autocorrelation(magnet_history)

        if np.isnan(correlation_time) or correlation_time <= 0:
            correlation_time = 1
        correlation_times.append((temperature, correlation_time))

        c = ising.heat_capacity(energy_history, temperature)
        specific_heat.append((temperature, c))
        specific_heat_error.append((temperature, ising.bootstrap_method(energy_history, 100, temperature, ising.heat_capacity)))
        
        magnet_samples = []
        energy_samples = []

        # Sample every two correlation times.
        for t in range(0, len(energy_history), 2 * np.ceil(correlation_time)):
            magnet_samples.append(magnet_history[t])
            energy_samples.append(energy_history[t])

        energy.append((temperature, np.mean(energy_samples)))
        energy_error.append((temperature, np.std(energy_samples)) / np.sqrt(len(energy_samples)))

        magnetization.append((temperature, np.mean(np.absolute(magnet_samples))))
        magnetization_error.append((temperature, np.std(np.absolute(magnet_samples)) / np.sqrt(len(np.absolute(magnet_samples)))))
    
    return energy, energy_error, magnetization, magnetization_error, specific_heat, specific_heat_error, correlation_times

In [ ]:
energy, energy_error, magnetization, magnetization_error, specific_heat, specific_heat_error, correlation_times = temperature_range(0.2, 5, 0.2, 4)

In [ ]:
print(specific_heat_error)
ising.plot_correlation_time_range(correlation_times, 4, "Magnetization")

ising.plot_quantity_range(energy, energy_error, "Internal Energy per Spin", 4)

ising.plot_quantity_range(magnetization, magnetization_error, "Magnetization per Spin", 4, exact=ising.exact_magnetization(0.2, 5))

ising.plot_quantity_range(specific_heat, specific_heat_error, "Specific Heat per Spin", 4)

In [ ]:
energy4, energy_error4, magnetization4, magnetization_error4, specific_heat4, specific_heat_error4, correlation_times4 = temperature_range(0.2, 5, 0.2, 4)
energy7, energy_error7, magnetization7, magnetization_error7, specific_heat7, specific_heat_error7, correlation_times7 = temperature_range(0.2, 5, 0.2, 7)
energy10, energy_error10, magnetization10, magnetization_error10, specific_heat10, specific_heat_error10, correlation_times10 = temperature_range(0.2, 5, 0.2, 10)

In [ ]:
ising.plot_correlation_time_range(correlation_times4, 4, "Magnetization", show_plot=False)
ising.plot_correlation_time_range(correlation_times7, 7, "Magnetization", show_plot=False)
ising.plot_correlation_time_range(correlation_times10, 10, "Magnetization")

ising.plot_quantity_range(energy4, energy_error4, "Internal Energy per Spin", 4, show_plot=False)
ising.plot_quantity_range(energy7, energy_error7, "Internal Energy per Spin", 7, show_plot=False)
ising.plot_quantity_range(energy10, energy_error10, "Internal Energy per Spin", 10, "lower right")

ising.plot_quantity_range(magnetization4, magnetization_error4, "Magnetization per Spin", 4, show_plot=False)
ising.plot_quantity_range(magnetization7, magnetization_error7, "Magnetization per Spin", 7, show_plot=False)
ising.plot_quantity_range(magnetization10, magnetization_error10, "Magnetization per Spin", 10, "upper right", exact=ising.exact_magnetization(0.2, 5))

ising.plot_quantity_range(specific_heat4, specific_heat_error4, "Specific Heat per Spin", 4, show_plot=False)
ising.plot_quantity_range(specific_heat7, specific_heat_error7, "Specific Heat per Spin", 7, show_plot=False)
ising.plot_quantity_range(specific_heat10, specific_heat_error10, "Specific Heat per Spin", 10)